<a href="https://colab.research.google.com/github/ido177/colab/blob/main/select/select_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.33MB [00:00, 54.4MB/s]


# Грузим БД

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
sqlite3.sqlite_version

'3.36.0'

In [3]:
con = sqlite3.connect('db')


In [4]:
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 4.44MB/s]


In [5]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [6]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [7]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

In [8]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

In [9]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

In [10]:
def select(sql):
  return pd.read_sql(sql,con)

# Селектим

In [15]:
sql = '''SELECT t.age * 3 AS age_mult3, 
    t.housing, 
    rc.housing, 
    us.job 
FROM german_credit AS t
LEFT JOIN russian credit AS rc 
    on ...
LEFT JOIN usa_credit AS uc 
    on ...
'''

Нужно избегать тупорылые псевдонимы, типа t1, t2 - можно запутаться\
as можно ставить по-желанию\
t.age * 3 as age_mult3 - умножаем значение на 3 и задаем новое значение колонки

In [16]:
select(sql)

,age,housing
0,33,own
1,43,own
2,52,own
3,35,own
4,28,own
...,...,...
995,65,free
996,30,own
997,33,own
998,29,own


In [23]:
sql = '''SELECT t.age * 3 AS age_mult3,
    t.age + t.credit_amount AS age_plus_amount,
    t.age * 1.0 / t.credit_amount AS age_div_amount
FROM german_credit AS t'''

умножение на единицу позволяет преобрести нужный формат, просто деление может быть нулем

In [24]:
select(sql)

,age_mult3,age_plus_amount,age_div_amount
0,99,3107,0.010735
1,129,1387,0.031994
2,156,988,0.055556
3,105,1428,0.025126
4,84,804,0.036082
...,...,...,...
995,195,2665,0.025000
996,90,4485,0.006734
997,99,6436,0.005154
998,87,5032,0.005797


In [38]:
sql = '''SELECT * FROM german_credit t 
WHERE t.contract_dt BETWEEN '2007-01-01' AND '2007-12-31'
--AND (t.purpose = 'car' OR t.purpose IN ('repairs'))

ORDER BY contract_dt desc, credit_amount
LIMIT 5
'''

df.head(5) тот же лимит\
desc - по убывающей, credit будет в данном случае по-возрастанию\
-- коммент в sql

In [39]:
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,31,female,1,own,little,None,1736,12,furniture/equipment,0,2007-12-30 13:29:15,995
1,35,male,1,own,little,None,2684,24,radio/TV,0,2007-12-30 00:40:33,256
2,23,female,2,rent,little,little,2406,30,furniture/equipment,1,2007-12-29 20:14:30,359
3,36,male,3,rent,None,moderate,7057,20,car,0,2007-12-29 16:10:08,99
4,30,male,2,own,little,moderate,639,12,repairs,1,2007-12-28 17:09:43,127
